In [79]:
import pandas as pd

# 读取鲜花特征数据
data = pd.read_excel('flowers_data.xlsx')

# 显示前几行数据
print(data.head())

   flower_id flower  适合送恋人  适合送长辈  适合送朋友  适合婚礼  适合葬礼  香气浓郁  花期长  红色  粉色  白色  \
0          1     玫瑰      1      0      1     1     0     1    1   1   1   1   
1          2    郁金香      1      0      1     1     0     0    0   1   1   1   
2          3     百合      1      1      1     1     0     1    1   1   1   1   
3          4    康乃馨      0      1      1     0     0     1    1   1   1   1   
4          5    向日葵      0      0      1     0     0     0    1   0   0   0   

   黄色  紫色  蓝色       花语  
0   1   1   0    爱情、浪漫  
1   1   1   0  爱的宣言、真爱  
2   1   1   0    纯洁、高贵  
3   1   1   0    母爱、温馨  
4   1   0   0    忠诚、阳光  


In [80]:
import os
import shutil
import random

# 设置图像文件夹路径
image_folder = 'images'

# 创建训练集、验证集和测试集的文件夹
os.makedirs('train', exist_ok=True)
os.makedirs('val', exist_ok=True)
os.makedirs('test', exist_ok=True)

# 遍历每种花的文件夹
for flower_dir in os.listdir(image_folder):
    flower_path = os.path.join(image_folder, flower_dir)
    if os.path.isdir(flower_path):
        images = os.listdir(flower_path)
        random.shuffle(images)
        
        # 划分图像
        train_split = int(0.6 * len(images))
        val_split = int(0.8 * len(images))
        
        train_images = images[:train_split]
        val_images = images[train_split:val_split]
        test_images = images[val_split:]
        
        # 创建目标目录
        os.makedirs(os.path.join('train', flower_dir), exist_ok=True)
        os.makedirs(os.path.join('val', flower_dir), exist_ok=True)
        os.makedirs(os.path.join('test', flower_dir), exist_ok=True)
        
        # 复制图像到相应的文件夹
        for img in train_images:
            shutil.copy(os.path.join(flower_path, img), os.path.join('train', flower_dir, img))
        for img in val_images:
            shutil.copy(os.path.join(flower_path, img), os.path.join('val', flower_dir, img))
        for img in test_images:
            shutil.copy(os.path.join(flower_path, img), os.path.join('test', flower_dir, img))

In [82]:
from fastai.vision.all import *

# 创建DataLoaders
dls = ImageDataLoaders.from_folder(path='.', train='train', valid='val', test='test', item_tfms=Resize(224))

# 创建模型
learn = cnn_learner(dls, resnet34, metrics=accuracy)

# 设置早停机制
early_stop = EarlyStoppingCallback(patience=3)

# 添加回调函数
learn.add_cb(early_stop)

# 训练模型
learn.fit_one_cycle(10)

# 保存模型
learn.export('flower_recognition_model.pkl')
# 打印训练后的准确度
val_accuracy = learn.validate()[1]
print(f'训练后的准确度: {val_accuracy:.2f}')

epoch,train_loss,valid_loss,accuracy,time
0,4.408186,2.973871,0.163441,01:08
1,3.716406,1.056412,0.731183,01:10
2,2.742634,0.424706,0.881720,01:09
3,2.062838,0.242536,0.926882,01:09
4,1.573848,0.161385,0.946237,01:07
5,1.233044,0.118817,0.969893,01:07
6,0.987560,0.095524,0.980645,01:08
7,0.806732,0.085590,0.984946,01:08
8,0.672693,0.082133,0.984946,01:07
9,0.574715,0.080790,0.984946,01:07


训练后的准确度: 0.98


In [83]:
# 使用测试集进行测试
test_dl = learn.dls.test_dl(get_image_files('test'))
preds, _, pred_labels = learn.get_preds(dl=test_dl, with_decoded=True)

In [84]:
# 计算测试集准确度
test_accuracy = accuracy(preds, pred_labels)
test_accuracy_value = test_accuracy.item()  # 转换为标量值
print(f'测试集准确度: {test_accuracy_value:.2f}')

测试集准确度: 1.00


In [86]:
# 特征选择模块
import pickle

# 特征选择函数
def recommend_flowers(features, data):
    # 计算匹配度
    data['match_score'] = data.iloc[:, 2:].sum(axis=1)
    # 按匹配度排序
    sorted_data = data.sort_values(by='match_score', ascending=False)
    return sorted_data

# 示例特征选择
selected_features = {
    '适合送恋人': 1,
    '适合送长辈': 0,
    '适合送朋友': 1,
    # 其他特征...
}

# 推荐鲜花
recommended_flowers = recommend_flowers(selected_features, data)

# 保存推荐结果
with open('recommended_flowers.pkl', 'wb') as f:
    pickle.dump(recommended_flowers, f)


C:\Users\ROG\AppData\Local\Temp\ipykernel_36656\1496503057.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  data['match_score'] = data.iloc[:, 2:].sum(axis=1)
